<a href="https://colab.research.google.com/github/RaulChihuaicura/Errores-1/blob/main/deserci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%pip install -U --quiet "numpy==2.0.2" "pandas==2.2.2" "scikit-learn==1.6.1"

In [ ]:
%pip install --quiet --no-deps "git+https://github.com/shankarpandala/lazypredict.git"

import os, signal; os.kill(os.getpid(), signal.SIGKILL)

  Preparing metadata (setup.py) ... done


In [1]:
import sys, numpy, pandas, sklearn
print("Python version:", sys.version)
print("NumPy version:", numpy.__version__)
print("Pandas version:", pandas.__version__)
print("Scikit-Learn version:", sklearn.__version__)

Python version: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
NumPy version: 2.0.2
Pandas version: 2.2.2
Scikit-Learn version: 1.6.1


In [2]:
from google.colab import files
up = files.upload()

Saving desercion_estudiantil_2000.csv to desercion_estudiantil_2000.csv


In [3]:
import pandas as pd
df = pd.read_csv("desercion_estudiantil_2000.csv")
df.head()

,id_estudiante,edad,genero,carrera,jornada,modalidad,horas_estudio_semana,sesiones_plataforma_semana,promedio_notas,asistencia_pct,retrasos_entregas,mensajes_soporte_mes,trabajo_part_time,beca,distancia_campus_km,dispositivos,velocidad_internet_mbps,ingresos_miles,satisfaccion_inicial,abandono
0,1,52,M,Salud,Vespertina,Presencial,10.5,7,6.56,74.3,2,6,No,No,12.1,1,101,657,3,0
1,2,37,F,Administracion,Diurna,Presencial,7.0,20,3.99,52.1,0,3,No,No,22.8,2,31,1044,5,1
2,3,40,M,Diseno,Vespertina,Presencial,8.7,12,5.05,61.6,2,5,Si,Si,9.0,2,116,1252,5,0
3,4,31,M,Salud,Diurna,Presencial,2.5,18,3.51,86.1,0,2,Si,Si,10.0,2,78,397,2,1
4,5,50,F,Ingenieria,Diurna,Presencial,8.5,17,4.97,90.5,1,0,No,Si,10.7,2,23,1087,4,1


In [8]:
from sklearn.model_selection import train_test_split

TARGET = "abandono" if "abandono" in df.columns else ("aprobado" if "aprobado" in df.columns else None)
assert TARGET is not None, f"No encuentro la columna objetivo. Columnas:{list(df.columns)[:30]}"

y = df[TARGET].copy()
if y.dtype == "0":
  y =y.replace({"Si": 1, "si": 1, 'SI':1, 'Yes':1, 'yes':1, 'YES':1, 'No':0, 'no':0, 'NO':0,'TRUE':1,'True':1,'true':1,'FALSE':0,'False':0,'false':0,'0':0}).astype(int)

else:
  y = y.astype(int)


In [9]:
X = df.drop(columns=[TARGET]+ [c for c in ['id', 'id_estudiante'] if c in df.columns])

In [10]:
X = pd.get_dummies(X, drop_first=False)

In [16]:
X_trval, X_test, y_trval, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X_trval, y_trval, test_size=0.2, random_state=42, stratify=y_trval)

In [18]:
import pandas as pd
X_tr = pd.concat([X_train, X_val], axis = 0)
y_tr = pd.concat([y_train, y_val], axis = 0)
y_tr.shape, X_test.shape, y_tr.shape, y_test.shape

((1600,), (400, 27), (1600,), (400,))

In [20]:
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models_df, predictions = clf.fit(X_tr, X_test, y_tr, y_test)
print("== LEADERBOARD (LazyClassifier) ===")
models_df.head(25)

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 766, number of negative: 834
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000559 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1380
[LightGBM] [Info] Number of data points in the train set: 1600, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.478750 -> initscore=-0.085051
[LightGBM] [Info] Start training from score -0.085051
== LEADERBOARD (LazyClassifier) ===


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
CalibratedClassifierCV,0.76,0.76,0.76,0.76,0.08
LinearSVC,0.76,0.76,0.76,0.76,0.03
LinearDiscriminantAnalysis,0.76,0.76,0.76,0.76,0.13
RidgeClassifierCV,0.76,0.76,0.76,0.76,0.02
RidgeClassifier,0.76,0.76,0.76,0.76,0.02
LogisticRegression,0.75,0.75,0.75,0.75,0.03
QuadraticDiscriminantAnalysis,0.75,0.75,0.75,0.75,0.05
GaussianNB,0.75,0.75,0.75,0.75,0.02
NearestCentroid,0.75,0.75,0.75,0.75,0.02


from matplotlib import pyplot as plt
_df_0['Accuracy'].plot(kind='hist', bins=20, title='Accuracy')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Balanced Accuracy'].plot(kind='hist', bins=20, title='Balanced Accuracy')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Time Taken']
  ys = series['F1 Score']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_11.sort_values('Time Taken', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Time Taken')
_ = plt.ylabel('F1 Score')

from matplotlib import pyplot as plt
_df_12['Accuracy'].plot(kind='line', figsize=(8, 4), title='Accuracy')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_13['Balanced Accuracy'].plot(kind='line', figsize=(8, 4), title='Balanced Accuracy')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_14['ROC AUC'].plot(kind='line', figsize=(8, 4), title='ROC AUC')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_15['F1 Score'].plot(kind='line', figsize=(8, 4), title='F1 Score')
plt.gca().spines[['top', 'right']].set_visible(False)

In [21]:
models_df.to_csv("Lazyclf_leaderboard.csv", index=False)
print("Guardado: Lazyclf_leaderboard.csv")

Guardado: Lazyclf_leaderboard.csv


In [22]:
import numpy as np, pandas as pd
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, confusion_matrix,
    f1_score, precision_score, recall_score, roc_auc_score
)

In [30]:
def _signal_auc(model, X):
    if hasattr(model, "predict_proba"):
        try:
            return model.predict_proba(X)[:,1]
        except Exception:
            pass
    if hasattr(model, "decision_function"):
        try:
            return model.decision_function(X)
        except Exception:
            pass
    return None

rows = []
binary = (len(np.unique(y_test)) == 2)
n_test = len(y_test)
for name, mdl in getattr(clf, "models", {}).items():

    try:
        y_pred = mdl.predict(X_test)
    except Exception:
        continue

    y_pred = np.asarray(y_pred).ravel().astype(int)
    if len(y_pred) != n_test:
        continue

    acc = accuracy_score(y_test, y_pred)
    bal = balanced_accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    auc = np.nan

    if binary:
        sig = _signal_auc(mdl, X_test)
        if sig is not None:
            try:
                auc = roc_auc_score(y_test, sig)
            except Exception:
                pass

    rows.append([name, acc, bal, f1, auc])

resumen_df = pd.DataFrame(rows, columns=["Modelo", "Accuracy", "Balanced Accuracy", "F1 Score", "AUC"])
resumen_df = resumen_df.sort_values(["Balanced Accuracy", "Accuracy"], ascending=False)
resumen_df.head(10)

,Modelo,Accuracy,Balanced Accuracy,F1 Score,AUC
3,CalibratedClassifierCV,0.76,0.76,0.75,0.83
12,LinearDiscriminantAnalysis,0.76,0.76,0.75,0.83
13,LinearSVC,0.76,0.76,0.75,0.83
21,RidgeClassifier,0.76,0.76,0.75,0.83
22,RidgeClassifierCV,0.76,0.76,0.75,0.83
14,LogisticRegression,0.75,0.75,0.75,0.83
19,QuadraticDiscriminantAnalysis,0.75,0.75,0.75,0.80
8,GaussianNB,0.75,0.75,0.74,0.82
15,NearestCentroid,0.75,0.75,0.75,0.83
0,AdaBoostClassifier,0.74,0.74,0.73,0.79
